In [1]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [2]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

## Covid 19 - A Comparison Between Number of Cases and PCR Tests

This dashboard displays a graphical comparison between the number of Covid-19 cases and PCR Tests in the UK. 

It is useful to observe the development of the Covid-19 Pandemic through these statistics. 


Select the button below to refresh data from [Public Health England](https://www.gov.uk/government/organisations/public-health-england).

In [3]:
with open("casesandtests.json", "rt") as INFILE:
    data=json.load(INFILE)  #This is the canned data in the json file. It has been assigned to a variable.

In [4]:
def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d") #This function allows the earliest and lastest date to be found 
#and then converted to the pandas type of representing dates. 

def wrangle_data(jsondata):
    """ Wrangle data and generate a dataframe  """
    #Firstly,actual data  stored as a list of dictionaries under the key'data' is retrieved:
    datalist=jsondata['data']
    dates=[dictionary['Date'] for dictionary in datalist ] #the dates are extracted and then sorted:
    dates.sort() #As the dates are written year-first, alphabetical ordering allows sorting. 
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
    index=pd.date_range(startdate, enddate, freq='D') #date_range is the date analog for a range of integers. It will include 
    #any missing dates from the list. Next the dataframe is defined:
    casesandtestsdf=pd.DataFrame(index=index, columns=['Cases', 'PCR Tests'])
    #The following code will proceed to fill the dataframe with data fetched: 
    for entry in datalist: 
        date=parse_date(entry['Date'])
        for column in ['Cases', 'PCR Tests']:
            if pd.isna(casesandtestsdf.loc[date, column]): 
            #Some data has a value of "None" which is replaced by 0: 
               value= float(entry[column]) if entry[column]!=None else 0.0
            #To acess a specific location in the dataframe:
               casesandtestsdf.loc[date, column]=value
#Any missing gaps due to missing dates are filled below: 
    casesandtestsdf.fillna(0.0, inplace=True)
    return casesandtestsdf
#The function is now created. We can call it again after refreshing the data through the API. 
#The JSON data should be called when the dashboard starts so the call is included below:
casesandtestsdf=wrangle_data(data) #data is the variable assigned to the canned JSON file


In [5]:
def access_api():
    """ Accesses the PHE API """
    filters = [
    'areaType=nation' 
    ] #The filter is defined first. According to the API documentation in the government website, this filter must be
      #used when accessing the metric newCasesByPublishDate.
    structure = {
    "Date": "date",
    "Cases": "newCasesByPublishDate",
    "PCR Tests": "newPCRTestsByPublishDate",
    }
    #The structure provides the desired statistics. It is possible to rename the keys of the dictionary to simpler names
    #as demonstrated above. The API then renames these fields when displaying the data
    
    api=Cov19API(filters=filters, structure=structure) #This accesses the API
    casesandtests=api.get_json() #This send a request to the API and retreives the data in JSON format
    return casesandtests
#This function will be called later on by the refresh button. 

In [6]:
def api_button_callback(button):
    """ Accesses API, wrangles data, updates global variable df used for plotting. """

    apidata=access_api() #here the function created above is called. This allows to get fresh data from API
  
 #The data is then wrangled and the dataframe is overwritten for plotting: 
    global casesandtestsdf
    casesandtestsdf=wrangle_data(apidata)

    refresh_graph() #This function allows the graph to refresh 

    apibutton.icon="check"
    apibutton.disabled=True

apibutton=wdg.Button(
    description='Refresh Data',
    disabled=False,
    button_style='info', 
    tooltip='Click to download current Public Health England data',
    icon='download'
)

apibutton.on_click(api_button_callback) 

display(apibutton)

Button(button_style='info', description='Refresh Data', icon='download', style=ButtonStyle(), tooltip='Click t…

The graph below shows the comparison between COVID-19 cases and PCR tests.

Interact with the graph to view it for one statistic only.  
You can also select the scale of the graph. The logarithmic scale is particularly useful to measure rates of change. 

In [7]:
#In this section, interactive controls are introduced to the dashboard:

def casesandtests_graph(gcols, gscale):   #This function accepts two parameters: a tuple of stats coming from the two widgets
                                          #below
    if gscale=='Linear Graph':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        casesandtestsdf[list(gcols)].plot(logy=logscale)
        plt.show() # important - graphs won't update if this is missing 
    #if the user hasn't selected an, the followinng message will appear:
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

#This widget allows for the selection of two displayed statistics 
series=wdg.SelectMultiple(
    options=['Cases', 'PCR Tests'], #these are the options available 
    value=['Cases', 'PCR Tests'], #these are the initial value 
    rows=2, #these are the number of rows for the selection box
    description='Select for:',
    disabled=False
)
#This widget allows for the selection of two types of graph:
scale=wdg.RadioButtons(
    options=['Linear Graph', 'Log Graph'],
    description='Select Scale:',
    disabled=False
)

controls=wdg.HBox([series, scale]) #the HBox widget allows the display of the two controls side by side

def refresh_graph():
    """ We change the value of the widget in order to force a redraw of the graph when data has been updated. """
    current=series.value
    if current==series.options[0]:
        other=(series.options[1],)
    else:
        other=(series.options[0],)
    series.value=other # This forces the redraw of the graph
    series.value=current # This now changes it back 
    

graph=wdg.interactive_output(casesandtests_graph, {'gcols': series, 'gscale': scale})
display(controls, graph)

Output()

Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).

Designed and developed by Maria Correia